In [1]:
%load_ext autoreload
#Wedutoreload 2

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')

In [2]:
df = pd.read_csv('../data/balanced_df.csv')

In [3]:
def filename_appender(df):
    df.fname = df.fname.apply(lambda x: '../img/' + x + '.jpg')
    return df

In [4]:
df = filename_appender(df)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD
keras = tf.keras
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [6]:
from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['code'] = LE.fit_transform(df['portrait'])

In [7]:
from sklearn.model_selection import train_test_split

X = df.drop('code',axis=1)

y = df.code

X_train, X_test, y_train,y_test = train_test_split(X,y, test_size=0.1)

X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.1)

In [8]:
train_filenames = [fname for fname in X_train.fname]
val_filenames = [fname for fname in X_val.fname]
test_filenames = [fname for fname in X_test.fname]

In [9]:
def reset_indices(arr):
    arr.reset_index(drop=True,inplace=True)
    return arr
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [150, 150])
    image /= 255.0
    return image

def remove_corrupt_files(filenames,y_arr):
    imgs = []
    y_arr_new_index = reset_indices(y_arr)
    
    
    for i,filename in enumerate(filenames):        
        # Try reading, decoding, resizing and normalizing images
        # Only appends files that do not cause any errors to an 'imgs' list
        try:
            img = load_and_preprocess_image(filename)
            imgs.append(np.asarray(img))  
        # If error occurs, skip to the next file and remove the file from the y array using its index
        except:
            
            del y_arr_new_index[i]
            continue
              
    X = np.asarray(imgs)
    y = np.asarray(y_arr_new_index)
    
    return X, y

In [10]:
X_train,y_train = remove_corrupt_files(train_filenames,y_train)
X_val,y_val = remove_corrupt_files(val_filenames,y_val)
X_test,y_test = remove_corrupt_files(test_filenames,y_test)

In [19]:
df[df.fname == train_filenames[0]]['portrait'].values[0]

'Constantius II'

In [35]:
import shutil, os


for filename in train_filenames:
    
    emperor = df[df.fname == filename]['portrait'].values[0]

    folder_path = f'../data/train_folder/{emperor}/'

    shutil.copy(filename, folder_path)

print('file transfer finished')

file transfer finished
